# 14_函数

作者说只要一段代码需要复制超过两次就应该考虑编写一个函数

In [1]:
## 什么时候应该使用函数

In [2]:
# 看一个例子

# library(tidyverse)
df <- tibble::tibble(
    a = rnorm(10),
    b = rnorm(10),
    c = rnorm(10),
    d = rnorm(10)
)

df$a <- (df$a - min(df$a, na.rm = T))/
    (max(df$a, na.rm = T) - min(df$a, na.rm = T))
df$b <- (df$b - min(df$b, na.rm = T))/
    (max(df$b, na.rm = T) - min(df$a, na.rm = T))
df$c <- (df$c - min(df$c, na.rm = T))/
    (max(df$c, na.rm = T) - min(df$c, na.rm = T))
df$d <- (df$d - min(df$d, na.rm = T))/
    (max(df$d, na.rm = T) - min(df$d, na.rm = T))



看出代码的作用和错误了吗，这就是复制容易引起的问题，我们编写一个函数来解决上述问题

In [3]:
x <- df$a
(x - min(x, na.rm = T))/
(max(x, na.rm = T) - min(x, na.rm = T))

[1] 0.8809228 1.0000000 0.3603719 0.6390122 0.0000000 0.8612409 0.7466528
 [8] 0.3227331 0.1250377 0.9291060

In [4]:
# 这里依旧有些重复，计算了３次最大值和最小值
# 我们学习一个函数range()
# help(range)
# range returns a vector containing the minimum and maximum of all the given arguments. 
rng <- range(x, na.rm = T)
(x- rng[1])/(rng[2] - rng[1])


[1] 0.8809228 1.0000000 0.3603719 0.6390122 0.0000000 0.8612409 0.7466528
 [8] 0.3227331 0.1250377 0.9291060

In [5]:
# 下面开始写函数
rescale01 <- function(x){
    rng <- range(x, na.rm = T)
    (x - rng[1]) / (rng[2] - rng[1])
}

rescale01(c(0:10))# 验证函数是否正确

[1] 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0

In [6]:
# 简化最初的代码
df$a <- rescale01(df$a)
df$b <- rescale01(df$b)
df$c <- rescale01(df$c)
df$d <- rescale01(df$d)
# 好多了，但依旧有重复，但我们这里先不优化，等学习的后面的知识再说


In [7]:
# 函数的优点是需求变化只需要改一处
x <- c(1:10, Inf)
rescale01(x)
# 能看出来包含无限值就不对了

[1]   0   0   0   0   0   0   0   0   0   0 NaN

In [8]:
# 我们修改一下函数
rescale01 <- function(x){
    rng <- range(x, na.rm = T, finite = T)
    (x - rng[1]) / (rng[2] - rng[1])
}
rescale01(x)

# 尽量写函数，不然就。。。

[1] 0.0000000 0.1111111 0.2222222 0.3333333 0.4444444 0.5555556 0.6666667
 [8] 0.7777778 0.8888889 1.0000000       Inf

## 练习

In [9]:
# 在rescale01()函数的第二个版本中，无穷大值未作任何处理。
# 重写rescale01()函数，将-Inf映射为0，Inf映射为1
y <- x
y[12] <- -Inf
rescale01 <- function(x){
    x[x==-Inf] = 0
     x[x == Inf] = 1
#     x
    rng <- range(x, na.rm = T, finite = T)
    (x - rng[1]) / (rng[2] - rng[1])
}
rescale01(y)

[1] 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0 0.1 0.0

编写一个函数来计算数值向量的方差和偏度

In [10]:
x <- 1:10
sum((x - mean(x))^2)/(length(x)-1)# 手动计算方差
sd(x)^2# 标准差的平方
var(x)# 方差
sqrt(var(x)/length(x))# 均值标准误
std.err <- function(x){
    sqrt(var(x)/length(x))
}# 这样也许会有小问题
y <- c(1,2,4,8,16)
std.err(y)
x <- 1:10
std.err(x)

[1] 9.166667

[1] 9.166667

[1] 9.166667

[1] 0.9574271

[1] 2.727636

[1] 0.9574271

In [11]:
std.err <- function(x){
    v <- var(x)
    n <- length(x)
    sqrt(v/n)
}# 这样可能好点
std.err(x)

[1] 0.9574271

我们计算一下偏度

公式：

$$\frac{\frac{1}{n-2}\sum_{i=1}^{n}(x_i-\bar{x})^3}{var(x)^{\frac{3}{2}}}$$

In [12]:
skenewness <- function(x){
    n <- length(x)
    m <- mean(x)
    v <- var(x)
    (1/(n-2))*sum((x-m)^3)/v^(3/2)
}# 计算偏度的函数
skenewness(rnorm(100))

[1] -0.007935023

编写 一个名为both_na()的函数，它接受两个长度相同的向量，如果两个向量中同一位置的值都是NA，则返回这样位置的数量。

In [13]:
both_na <- function(a,b){
# a <- c(1,NA,2,NA,3)
# b <- c(NA,NA, 1, NA,3)# 测试
if(length(a)==length(b)){
j=0
for(i in 1:length(a)){
    if(is.na(a[i]) & is.na(b[i])){
         print(i)
         j = j + 1
    }
}
print(j)}else{print("length a is not = b")}
}# 暂时能用，不过循环用for怕是会降低速度，数据稍微大一点就慢了

a <- c(1,NA,2,NA,3)
b <- c(NA,NA, 1, NA,3)
both_na(a,b)

[1] 2
[1] 4
[1] 2


以下函数的功能是什么？尽管它们都很简短，但是用处都很大，为什么？

is_directory <-function(x) file.info(x)$isdir 

is_readable <-function(x) file.access(x, 4) ==0

从名字就可以看出来，第一个是判断是不是目录，第二个是判断文件是不是可读的，接受的输入是字符串向量，判断文件的读、写、可执行权限再windows下没什么用，主要是再linux下才用。

 阅读以下3个函数的源代码，推测出它们的功能，然后使用头脑风暴给它们以更好的名称

In [14]:
f1 <-function(string, prefix) { 
    substr(string, 1, nchar(prefix)) == prefix 
}

# x <- "function"
# y <- "fun"
# f1(x,y)

f2 <-function(x) { 
    if (length(x) <=1) return(NULL)   
    x[-length(x)] 
}
# x <- c(1:3)
# y <- c("a","b")
# f2(x)
# f2(y)

f3 <-function(x, y) { 
    rep(y, length.out =length(x)) 
}
# x <- c(1:8)
# y <- c("a","b")
# f3(x,y)

## 条件执行

In [15]:
# 用if描述一个向量的各个元素是否被命名的函数
# 查看if的帮助要加反引号
has_name <- function(x) {
    nms <- names(x)
    if(is.null(nms)){
        rep(FALSE, length(x))
    } else{
        !is.na(nms) & nms != ""
    }
}
# x <- c("a"=1,"b"=2)
# x
# is.vector(x)
# y <- 1:8
# has_name(x)
# has_name(y)



***if条件注意***

if的条件要么是真，要么是假，不能是向量和NA，

可以使用||（或）和&&（与）操作符来组合多个逻辑表达式。

?all# Given a set of logical vectors, are all of the values true? 

?any # Given a set of logical vectors, is at least one of the values true? 

identical()非常严格，总是返回一个TRUE或者一个FALSE，并且不限制参数类型

浮点数比较使用dplyr::near()函数


多重条件
```
if (this) {  
    # 做一些操作
} else if (that) {  
    # 做另外一些操作
} else {   
    # 
}
```
但如果你有一长串if语句，那么就要考虑重写了。重写的一种方法是使用switch()函数，它先对第一个参数求值，然后按照名称或位置在后面的参数列表中匹配返回结果：
```{R}
function(x, y, op) { 
  switch(op,
    plus = x + y, 
    minus = x - y, 
    times = x * y, 
    divide = x / y, 
    stop("Unknown op!") 
  ) 
}
```
可以重写一长串if语句的另一个函数是cut()，它可以将连续变量离散化

代码风格就是不要写的太乱七八糟，一定要该空格空格，该缩进缩进，该换行换行，看别人怎么写的，自己也怎么写

前面没有做示范，看练习

## 练习

In [16]:
# (1) if与ifelse()的区别是什么？仔细阅读帮助文档，然后构建3个示例，说明它们之间的关键区别。
# 直接看一个例子
a <- 3
b <- 4
a <- 9

if(a>b){
    print(a)
}else{
    print(b)
}
ifelse(a>b, a, b)


ifelse(a>b, ifelse(a<7, a, "top"), b)
if(a>b){
    if (a<7){
    print(a)
}else{
    print("top")
}
}else{ 
    print(b)
}

# 很明显ifelse要简洁

[1] 9


[1] 9

[1] "top"

[1] "top"


In [17]:
# 编写一个欢迎函数，根据每天的不同时间输出“上午好”“下午好”和“晚上好”。（提示：使用lubridate::now()函数默认的时间参数，这会使得函数测试更容易一些。）
hello <- function(){
    now_time <- lubridate::now()
    h <- lubridate::hour(now_time)
    ifelse(h > 7 , 
           ifelse(h < 13, "good morning", 
                          ifelse(h < 20, "good afternoon", 
                                 "good evening")), "good evening")
}# 此函数不需要输入参数
hello()

[1] "good evening"

In [18]:
# 实现fizzbuzz函数，接受一个数值作为输入。如果这个数值能被3整除，那么就返回“fizz”；如果能被5整除，就返回“buzz”；如果能同时被3和5整除，则返回“fizzbuzz”；否则，就返回这个数值。
fizzbuzz <- function(x){
    if(!x%%3 && !x%%5){
        print("fizzbuzz")
    }else if(!x%%3){
        print("fizz")
    }else if(!x%%5){
        print("buzz")
    }else{
         print(x)
    }
}# 逻辑不太好

x <- c(2,6,7,10,15)
for (i in x){
    fizzbuzz(i)
    }
    
print(rep("-",16))
fizzbuzz <- function(x){
    z <- ifelse(!x%%3, ifelse(!x%%5, "fizzbuzz", "fizz"), 
           ifelse(!x%%5, "buzz", x))
    print(z)
    }# 简洁
# y <- 11
# fizzbuzz(y)
for (i in x){
    fizzbuzz(i)
    }

[1] 2
[1] "fizz"
[1] 7
[1] "buzz"
[1] "fizzbuzz"
 [1] "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-" "-"
[1] 2
[1] "fizz"
[1] 7
[1] "buzz"
[1] "fizzbuzz"


如何使用cut()函数来简化以下这段嵌套if-else语句？
```
if (temp <=0) { 
    "freezing" 
    } else if (temp <=10) { 
    "cold" 
    } else if (temp <=20) { 
    "cool" } else if (temp <=30) { 
    "warm" 
    } else { 
    "hot"
    }
```

In [19]:
feel_temp <- function(temp){
    cut(temp, c(-100,0,10,20,30,100), right = T, 
    labels = c("freezing", "cold", "cool", "warm", "hot"))
}
feel_temp(35)
feel_temp(c(-15,22))
# 使用cut确实优雅许多，还可以接受多个值，如果给的参数越界就会出错

[1] hot
Levels: freezing cold cool warm hot

[1] freezing warm    
Levels: freezing cold cool warm hot

## 函数参数

函数参数如果设置默认值一定要设置最常用的值，
比如：

In [20]:
# 使用近似正态分布计算均值两端的置信区间
mean_ci <- function(x, conf = 0.95){
    se <- sd(x) / sqrt(length(x))
    alpha <- 1 - conf
    mean(x) + se * qnorm(c(alpha/2, 1-alpha/2))
}

x <- runif(100)
mean_ci(x)
mean_ci(x, conf = 0.99)
# 在调用函数时，我们经常省略数据参数的名称，
# 因为其使用太普遍了。如果不想使用细节参数的默认值，
# 那么你应该使用细节参数的完整名称

[1] 0.4553644 0.5614450

[1] 0.4386979 0.5781115

### 检查参数值

当编写的函数越来越多时，你有时会记不清某个函数到底是用来做什么的。这时就很容易使用无效的参数来调用函数。为了解决这种问题，应该对函数参数进行明确的限制。

In [21]:
wt_mean <-function(x, w) { 
    sum(x * w) /sum(x) 
}

wt_var <-function(x, w) {
    mu <-wt_mean(x, w) 
    sum(w * (x - mu) ^ 2) /sum(w)                                                          
} 

wt_sd <-function(x, w) { 
    sqrt(wt_var(x, w)) 
}

# 这三个小函数就不解释了
# 如果x和w的长度不一样，那么会发生什么情况？
wt_mean(1:6, 1:3)
# 这种情况下，由于R的向量循环机制，代码不会出错

[1] 2.190476

In [22]:
# 检查参数，在必要时抛出错误

wt_mean <- function(x, w){
    if (length(x) != length(w)){
        stop("'x' and 'w' must be the same length", call.=F)
    }
    sum(w * x) / sum(x)
}

# wt_mean(1:6, 1:3)

也别做的太过分了

例如，如果你还需要添加一个na.rm参数，那么我们大概不会检查得如此仔细：
```
wt_mean <-function(x, w, na.rm =FALSE) { 
    if (!is.logical(na.rm)) { 
    stop("`na.rm` must be logical")   
    } 
    if (length(na.rm) !=1) { 
    stop("`na.rm` must be length 1")   
    } 
    if (length(x) !=length(w)) { 
    stop("`x` and `w` must be the same length", call. =FALSE)
    } 
    if (na.rm) {     
    miss <-is.na(x) |is.na(w)     
    x <- x[!miss]     
    w <- w[!miss]   
    } 
    sum(w * x) /sum(x) 
    }
```
这就有点事倍功半，得不偿失了。

In [23]:
# 一个折中方案是使用内置的stopifnot()函数

wt_mean <-function(x, w, na.rm =FALSE) { 
    stopifnot(is.logical(na.rm), length(na.rm) == 1) 
    stopifnot(length(x) ==length(w)) 
    if (na.rm) {     
        miss <-is.na(x) |is.na(w)     
        x <- x[!miss]     
        w <- w[!miss]                                                                                                                                
    } 
    sum(w * x) /sum(x) 
} 

# wt_mean(1:6, 6:1, na.rm ="foo") 

In [24]:
# 任意数量输入参数(...)

# 这个参数的作用非常大，因为你可以将它捕获的值传给另一个函数。
# 如果你的函数是另一个函数的包装器，那么这种一网打尽的方式就非常有用了。
# 例如，我们经常用以下方式创建辅助函数来包装str_c()函数：
commas <-function(...) stringr::str_c(..., collapse =", ") 
commas(letters[1:10]) 
# 这个函数很简单就不说了


[1] "a, b, c, d, e, f, g, h, i, j"

In [25]:
# 下面这个函数可以提一下

rule <- function(..., pad = "-"){
    title <- paste0(...)
    width <- getOption("width") - nchar(title) - 5
    cat(title, " ", stringr::str_dup(pad, width), "\n", sep = "")
}
# getOption()函数获得宽度，nchar()获得字符串长度
# str_dup()的两个参数分别是重复内容和重复次数
# 用这个函数我们就可以方便的在输出中设置分割线了
rule("Important Line")
rule("分割线", pad = "*")

Important Line -------------------------------------------------------------
分割线 ************************************************************************


In [26]:
# 函数调用commas(letters, collapse = "-")的作用是为了设置分隔符为“-”，
# 不过不行，需要改函数

# 如果能为pad参数提供多个字符，那真是太好了，例如rule("Title", pad = "-+")。
# 为什么现在的rule()函数还做不到这一点？应该如何改进？

rule <- function(..., pad = "-"){
    title <- paste0(...)
    width <- getOption("width") - nchar(title) - 5
    cat(title, " ", stringr::str_dup(pad, floor(width/nchar(pad))), "\n", sep = "")
}
# 这里通过计算pad参数的字符个数，向下取整计算重复次数
# 顺便记一下取整函数
# R里面数字取整相关的操作有一组函数：
# ceiling ；floor；trunc； round； signif
# ceiling:向上取整；floor:向下取整；trunc:向零取整；round:四舍五入；signif:科学计算保留有效位数
# 不一个个试了

rule("我是分割线", pad = "%*")
# 剩下两道练习自己看帮助或者上网搜吧

我是分割线 %*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*%*


## 返回值和环境

可以通过return()语句提前返回一个值，也不要使用太多，小例子就不写了。尽量让自己的函数支持管道操作，其他看书。

不要重定义一些函数和运算，这样容易出问题，理论看书，没多少写的。